### [미션]
1. 제주도 지식인 검색을 통해 검색 결과에 대한 url 100개 취득
2. 100개의 url에서 질문/답변 각각 크롤링 진행
3. 크롤링 결과는 질문과 답변 각각 저장
4. 질문에 대한 워드클라우드 제작
5. 답변에 대한 워드클라우드 제작

In [ ]:
#글꼴 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 글꼴 설치후에는 반드시 런타임 재실행

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/py_pro

1. 제주도 검색 결과에 대한 url 추출하기

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import sys

In [ ]:
# 검색어와 페이지 번호를 이용해 원하는 웹 사이트 정보 가져와 파싱후 리턴하는 함수
def url_res(sch_txt, page=1):
    url='https://kin.naver.com/search/list.naver?query='+ sch_txt + '&page=' + str(page)

    html=requests.get(url)
    if html.status_code != 200:
        sys.exit('페이지 정보를 가져오지 못했습니다.')
    
    soup = bs(html.content, "html.parser")    # 데이터 파싱

    return soup

In [ ]:
# 검색한 결과를 페이지별로 이동하면서 100개의 URL 추출

sch_txt=input('검색어 입력: ')
link_lst=[]   # 링크 URL 저장 변수 생성

for i in range(1, 11): 
    soup=url_res(sch_txt, i)    # url_res() 함수에서 파싱 결과 받아오기
    
    uls_soup=soup.find('ul', class_="basic1")  # 본문 내용 전체 추출
    dts_soup=uls_soup.find_all('dt')           # 각 항목별 제목 추출

    for dt in dts_soup:
        link=dt.find("a")['href']
        link_lst.append(link)

print(len(link_lst))

In [ ]:
# 지식인 질문 내용 가져오기
import re

quest_lst=[]     # 질문에 대한 저장 공간
answer_lst=[]    # 답변에 대한 저장 공간

for url in link_lst:
    # 하위 링크에서 정보 받아오기
    html=requests.get(url)
    if html.status_code != 200:
        continue

    soup=bs(html.content, 'html.parser')

    # 질문 가져오기
    try:
        quest_val=soup.find('div', class_="c-heading__content").get_text()
        quest_val=re.sub('[^ㄱ-ㅣ가-힣a-zA-Z0-9 ]', "", quest_val)
        quest_lst.append(quest_val)
    except:
        print('-')

    # 답변 가져오기
    try:
        answer_soup=soup.find_all('div', class_="se-module se-module-text")
        for asw in answer_soup:
            answer_val=asw.get_text()
            answer_val=re.sub('[^ㄱ-ㅣ가-힣a-zA-Z0-9 ]', "", answer_val)
            answer_lst.append(answer_val)
    except:
        print('-')


#quest_lst
#answer_lst

In [ ]:
!pip install konlpy

In [ ]:
# 질문에 대한 형태소 분석
import nltk
from konlpy.tag import Okt
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='NanumBarunGothic')

okt=Okt()

quest_nouns=[]
answer_nouns=[]
for txt in quest_lst:
    quest_nouns += okt.nouns(txt)

for txt in answer_lst:
    answer_nouns += okt.nouns(txt)

In [ ]:
cov_ko=nltk.Text(quest_nouns, name='test')

plt.figure(figsize=(12, 6))
cov_ko.plot(50)
plt.show()

In [ ]:
cov_ko=nltk.Text(answer_nouns, name='test')

plt.figure(figsize=(12, 6))
cov_ko.plot(50)
plt.show()

In [ ]:
from collections import Counter   # jdk에서 제공하는 라이브러리

# 질문/답변에 대한 단어 빈도수
quest_cnt=Counter(quest_nouns)        # 동일 문자수를 계산하는 함수
answer_cnt=Counter(answer_nouns)

In [ ]:
# 데이터 구조 변경(튜플형)
rank_quest=quest_cnt.most_common()
rank_answer=answer_cnt.most_common()

In [ ]:
# 단어 빈도수가 10개 미만인 데이터 모두 삭제
rank_quest_rev=[]
for (keyword, num) in rank_quest:
    if num >= 10 :
        rank_quest_rev.append((keyword, num))

rank_answer_rev=[]
for (keyword, num) in rank_answer:
    if num >= 10 :
        rank_answer_rev.append((keyword, num))

In [ ]:
# 워드클라우드 설치
!pip install WordCloud

In [ ]:
# 워드클라우드 생성
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 글꼴 패치 설정
path='/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf'

#============== 질문에 대한 워드클라우드 생성 ==========================

# rank_quest_rev을 딕셔너리 구조의 데이터로 변경
data=dict(rank_quest_rev)

# 워드클라우드 생성하기
wc1=WordCloud(font_path=path, relative_scaling=0.2, background_color='white')
wc2=wc1.generate_from_frequencies(data)

plt.figure(figsize=(10, 10))
plt.imshow(wc2)     # 워드클라우드 이미지 출력
plt.axis('off')     # 눈금선 제거
plt.show()          # 화면에 표시
print("\n\n")
#============== 답변에 대한 워드클라우드 생성 ==========================

# rank_answer_rev을 딕셔너리 구조의 데이터로 변경
data=dict(rank_answer_rev)

# 워드클라우드 생성하기
wc1=WordCloud(font_path=path, relative_scaling=0.2, background_color='white')
wc2=wc1.generate_from_frequencies(data)

plt.figure(figsize=(10, 10))
plt.imshow(wc2)     # 워드클라우드 이미지 출력
plt.axis('off')     # 눈금선 제거
plt.show()  